# Import library

In [10]:
import numpy as np  #multidimensional arrays
import cv2   #process images and videos to identify objects, faces
#keras provides high-level APIs used for easily building and training models
from keras.models import Sequential # arranging the Keras layers in a sequential order
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from keras.utils.vis_utils import plot_model
import os

For dataset :https://www.kaggle.com/msambare/fer2013?
In dataset there are two floder like test data and train data.Both test and tarin  floder there are seven floder like that angry,digust,fear,happy,natural,sad and surprise 

In [12]:
train_dir = 'train'
val_dir = 'test'
img_size=48
batch_size=64
train_datagen = ImageDataGenerator(rescale=1./255) #allows you to randomly rotate images through any degree between
#0 and 360 by providing an integer value in the rotation_range argument.
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size,img_size),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')
val_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(img_size,img_size),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [13]:
for i in os.listdir("train/"):
    print(str(len(os.listdir("train/"+i))) +" "+ i +" images")

3995 angry images
436 disgust images
4097 fear images
7215 happy images
4965 neutral images
4830 sad images
3171 surprise images


In [14]:
for i in os.listdir("test/"):
    print(str(len(os.listdir("test/"+i))) +" "+ i +" images")

958 angry images
111 disgust images
1024 fear images
1774 happy images
1233 neutral images
1247 sad images
831 surprise images


In [15]:
emotion_model = Sequential()
'''The rectified linear activation function or ReLU for short is a piecewise linear function that will output the
input directly if it is positive, otherwise, it will output zero.'''
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))#output=(48-3+0)/1+1=46
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))#output=(46-3+0)/1+1=44
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#output=devided input by 2 it means 22,22,64
emotion_model.add(Dropout(0.25))#reduce 25% module at a time of output
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',input_shape=(48,48,1)))#(22-3+0)/1+1=20
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#10
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))#(10-3+0)/1+1=8
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#output=4
emotion_model.add(Dropout(0.25))#nothing change
emotion_model.add(Flatten())#here we get multidimention output and pass as linear to the dense so that 4*4*128=2048
emotion_model.add(Dense(1024, activation='relu'))#hddien of 1024 neurons of input 
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))#hddien of 7 neurons of input
plot_model(emotion_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)#save model leyer as model_plot.png
emotion_model.summary()

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                  

In [8]:
from keras import callbacks
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)
  
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=25,
        validation_data=validation_generator,
        validation_steps=7178 // 64,
        callbacks =[earlystopping]) # find optimal no. of epochs

Epoch 1/25
448/448 [==============================] - 220s 489ms/step - loss: 1.3639 - accuracy: 0.4810 - val_loss: 1.3198 - val_accuracy: 0.4990
Epoch 2/25
448/448 [==============================] - 230s 513ms/step - loss: 1.3188 - accuracy: 0.4993 - val_loss: 1.2998 - val_accuracy: 0.5057
Epoch 3/25
448/448 [==============================] - 232s 518ms/step - loss: 1.2810 - accuracy: 0.5145 - val_loss: 1.2747 - val_accuracy: 0.5130
Epoch 4/25
448/448 [==============================] - 235s 525ms/step - loss: 1.2440 - accuracy: 0.5270 - val_loss: 1.2409 - val_accuracy: 0.5250
Epoch 5/25
448/448 [==============================] - 238s 532ms/step - loss: 1.2142 - accuracy: 0.5402 - val_loss: 1.2122 - val_accuracy: 0.5333
Epoch 6/25
448/448 [==============================] - 228s 510ms/step - loss: 1.1864 - accuracy: 0.5539 - val_loss: 1.1929 - val_accuracy: 0.5424
Epoch 7/25
448/448 [==============================] - 239s 533ms/step - loss: 1.1552 - accuracy: 0.5652 - val_loss: 1.1767 -

# Saving a model using save_weights

In [9]:
emotion_model.save_weights('model.h5')

# Open webcam using openCV

In [16]:
# start the webcam feed
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Natural", 5: "Sad", 6: "Surprised"}

#cap = cv2.VideoCapture('facial_exp.mkv')
cap = cv2.VideoCapture(0)
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    #frame = cv2.flip(frame, 1)
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(600,500),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()